In [23]:
pip install hazm

In [24]:
pip install transformers

In [25]:
import pandas as pd
import regex as re
from hazm import Normalizer, WordTokenizer
from transformers import BertForSequenceClassification, BertTokenizer
from concurrent.futures import ThreadPoolExecutor


In [26]:

# Load your Persian dataset as a DataFrame, assuming it has a 'Text' column
# Replace 'your_dataset.csv' with the actual path to your dataset CSV file
df = pd.read_csv("/content/final_4407 copy.csv" ,encoding='utf-8-sig')



In [27]:
df.head(2)

,number,Text,Followers Count,Friends Count,Statuses Count,FAKE_label,sentiment
0,2,@Gandalfkhaki ویروس عرفانی کرونا میخوره تو صور...,2631,3328,28879,1,1
1,3,خوبی کرونا که یک ویروس کشنده است اینه که شما ا...,1009,5000,640,1,1


In [28]:
# Load Persian stop words (if needed)
# Replace '/path/to/persian_stop_words.txt' with the actual path to your stop words file
stop_words = []
with open('/content/drive/MyDrive/persian_stop_words.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')



In [29]:
# Load the pre-trained ParsBERT sentiment analysis model and tokenizer
model_name = "nimaafshar/parsbert-fa-sentiment-twitter"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, from_tf=True)






All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [30]:
from hazm import Normalizer, WordTokenizer
import re

# Load Persian stop words
stop_words = []
with open('/content/drive/MyDrive/persian_stop_words.txt', 'r') as file:
    stop_words = file.read().split('\n')

# Define a function to preprocess text
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove punctuation marks and English numbers
    text = re.sub(r'[a-zA-Z0-9]+', '', text)
    text = re.sub(r'[^\w\s0-9a-zA-Z]+', '', text)

    # Remove mentions of user screen names
    text = re.sub(r'@\w+', '', text)

    # Remove underscores
    text = text.replace('_', '')

    return text

# Batch process the texts
batch_size = 100  # Adjust the batch size as needed
num_batches = (len(df) // batch_size) + 1
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    df.loc[start_idx:end_idx, 'Text'] = df.loc[start_idx:end_idx, 'Text'].apply(preprocess_text)

# Tokenize the text and remove stop words
#tokenizer = WordTokenizer()
#df['Tokens'] = df['Text'].apply(lambda text: [word for word in tokenizer.tokenize(text) if word not in stop_words])

# Print the DataFrame to check the results
print(df.head())


   number                                               Text  Followers Count  \
0       2              ویروس عرفانی کرونا میخوره تو صورتشون              2631   
1       3  خوبی کرونا که یک ویروس کشنده است اینه که شما ا...             1009   
2       4   از سوپ آمده تا پیتزا یا خمیر دندان فرقی نداره...               55   
3       5  همانطور که جامعه جهانی اجماع کردن و جلوی ویروس...              414   
4       6                   طراحی ویروس کرونا برای ژن ایرانی               22   

   Friends Count  Statuses Count  FAKE_label  sentiment  
0           3328           28879           1          1  
1           5000             640           1          1  
2            260             240           1          1  
3            454           12074           1          1  
4            112            1925           1          0  


In [31]:
# Preprocess the 'Text' column of the DataFrame
df['Preprocessed_Text'] = df['Text'].apply(preprocess_text)




In [32]:
df['Preprocessed_Text'].head(3)

0                ویروس عرفانی کرونا میخوره تو صورتشون 
1    خوبی کرونا که یک ویروس کشنده است اینه که شما ا...
2     از سوپ آمده تا پیتزا یا خمیر دندان فرقی نداره...
Name: Preprocessed_Text, dtype: object

In [33]:
# Function to analyze sentiment for a batch of texts
def analyze_sentiment_batch(texts):
    sentiments = []
    for text in texts:
        # Tokenize the text and prepare input for the model
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # Perform sentiment analysis using ParsBERT
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = logits.argmax().item()

        sentiments.append(predicted_label)
    return sentiments


In [34]:

# Split the preprocessed texts into batches
batch_size = 32  # Adjust the batch size as needed
batches = [df['Preprocessed_Text'][i:i+batch_size] for i in range(0, len(df), batch_size)]


In [35]:
# Process batches in parallel to analyze sentiment
sentiments = []
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
    results = executor.map(analyze_sentiment_batch, batches)
    for result in results:
        sentiments.extend(result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [36]:
# Add the sentiment labels to the DataFrame
df['Sentiment_Label'] = sentiments


In [37]:
df['Sentiment_Label'].head(3)

0    1
1    0
2    0
Name: Sentiment_Label, dtype: int64

In [38]:
# Save the DataFrame with predicted sentiments to a new CSV file
output_csv_path = "/content/output_4407.csv"  # Replace with your desired output file path
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

In [39]:
from google.colab import files
files.download('/content/output_4407.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>